In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

drive  sample_data


In [ ]:
# !ls nnUNet_Workspace/nnUNet/

In [ ]:
# ----------------------------------------------------------------------
# BƯỚC 2: CÀI ĐẶT MÔI TRƯỜNG & GIẢI NÉN CODE
# ----------------------------------------------------------------------
import os
import shutil

In [ ]:
# Tạo thư mục làm việc
!mkdir -p /content/nnUNet_Workspace
%cd /content/nnUNet_Workspace

# Copy và giải nén Code (Giả sử bạn để file zip ở thư mục gốc Drive/BraTS2020_EDL_Project)
!cp "/content/drive/MyDrive/XUM_project/nnUNet.zip" .
!unzip -q nnUNet.zip
!rm nnUNet.zip

# Cài đặt thư viện từ source code vừa giải nén
%cd nnUNet
!pip install -e .
!pip install --upgrade hiddenlayer graphviz # Cài thêm visualizer nếu cần

/content/nnUNet_Workspace
/content/nnUNet_Workspace/nnUNet
Obtaining file:///content/nnUNet_Workspace/nnUNet
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
!ls

drive  nnUNet_Workspace  sample_data


## Giải nén `nnUNet_preprocessed` vào thư mục content của Colab

In [ ]:
import os
import zipfile
from tqdm import tqdm
import shutil

# ----------------------------------------------------------------------
# BƯỚC 3: GIẢI NÉN DỮ LIỆU PREPROCESSED – AUTO FIX FOLDER DOUBLE-LAYER
# ----------------------------------------------------------------------

os.environ['nnUNet_raw'] = "/content/nnUNet_raw"
os.environ['nnUNet_preprocessed'] = "/content/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/content/drive/MyDrive/XUM_project/nnUNet_results"

!mkdir -p $nnUNet_raw
!mkdir -p $nnUNet_preprocessed

zip_path_drive = "/content/drive/MyDrive/XUM_project/nnUNET_preprocessed.zip"
zip_local = "/content/nnUNET_preprocessed.zip"
extract_root = "/content/nnUNet_preprocessed"

print("📦 Copy file từ Drive…")
!cp "$zip_path_drive" "$zip_local"

print("💨 Copy xong. Bắt đầu giải nén…")

with zipfile.ZipFile(zip_local, 'r') as zip_ref:
    # Detect folder root name inside zip (vd: nnUNET_preprocessed/)
    root_name = zip_ref.namelist()[0].split('/')[0]

    file_list = zip_ref.infolist()
    total_files = len(file_list)

    print(f"🔧 Phát hiện folder root trong ZIP: {root_name}")
    print("🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…")

    for file in tqdm(file_list, total=total_files, desc="🔄 Unzipping"):
        file_path = file.filename

        # Bỏ lớp folder đầu tiên
        fixed_path = file_path.replace(root_name + "/", "", 1)

        if fixed_path.strip() == "":
            continue

        # Chỉ định nơi lưu đúng
        target_path = os.path.join(extract_root, fixed_path)

        # Tạo thư mục nếu cần
        if file.is_dir():
            os.makedirs(target_path, exist_ok=True)
        else:
            os.makedirs(os.path.dirname(target_path), exist_ok=True)
            with zip_ref.open(file) as source, open(target_path, "wb") as target:
                target.write(source.read())

os.remove(zip_local)

print("🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥")

📦 Copy file từ Drive…
💨 Copy xong. Bắt đầu giải nén…
🔧 Phát hiện folder root trong ZIP: nnUNET_preprocessed
🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…


🔄 Unzipping: 100%|██████████| 2584/2584 [03:09<00:00, 13.64it/s]


🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥


In [ ]:
!ls /content/drive/MyDrive/XUM_project

inference_results	 nnUNET_raw.zip  run_batch_inference.py
inference_results_batch  nnUNet_results  Train_BraTS_EDL.ipynb
nnUNET_preprocessed.zip  nnUNet.zip


## KIỂM TRA BIẾN MÔI TRƯỜNG & DATA

In [ ]:
print(f"Raw: {os.environ['nnUNet_raw']}")
print(f"Preprocessed: {os.environ['nnUNet_preprocessed']}")
print(f"Results: {os.environ['nnUNet_results']}")

# Kiểm tra file EDLTrainer có tồn tại không
if os.path.exists("/content/nnUNet_Workspace/nnUNet/nnunetv2/training/nnUNetTrainer/EDLTrainer.py"):
    print("✅ Tim thay file EDLTrainer.py!")
else:
    print("❌ KHONG TIM THAY EDLTrainer.py! Kiem tra lai file zip code.")

Raw: /content/nnUNet_raw
Preprocessed: /content/nnUNet_preprocessed
Results: /content/drive/MyDrive/XUM_project/nnUNet_results
✅ Tim thay file EDLTrainer.py!


In [ ]:
!ls nnUNet_preprocessed -a

.  ..  Dataset101_BraTS2020


### Training bằng `EDLTrainer`

In [ ]:
# ----------------------------------------------------------------------
# BƯỚC 5: BẮT ĐẦU TRAINING (The Moment of Truth)
# ----------------------------------------------------------------------
# Lệnh train:
# Dataset101_BraTS2020: Tên dataset
# 3d_fullres: Cấu hình
# 0: Fold (thường chạy fold 0 trước)
# -tr EDLTrainer: Dùng Trainer Custom của bạn
# --npz: Lưu xác suất softmax (để dùng cho validation sau này)

# Khai báo biến môi trường trực tiếp trên cùng 1 dòng lệnh để đảm bảo nnU-Net nhận diện được
!nnUNet_raw="/content/nnUNet_raw" \
nnUNet_preprocessed="/content/nnUNet_preprocessed" \
nnUNet_results="/content/drive/MyDrive/XUM_project/nnUNet_results" \
nnUNetv2_train Dataset101_BraTS2020 3d_fullres 0 -tr EDLTrainer --npz


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-11-22 13:01:22.781290: Using torch.compile...
2025-11-22 13:01:27.235220: do_dummy_2d_data_aug: False
2025-11-22 13:01:27.240783: Creating new 5-fold cross-validation split...
2025-11-22 13:01:27.247902: Desired fold for training: 0
2025-11-22 13:01:27.252467: This split has 294 training an